## Importing modules and reading csv file

In [1]:
import os
import polars as pl
from utils import *
import pandas as pd
from datetime import datetime

In [28]:
def get_cleaned_df(state):
    csv_files_dir = '../data/cleaned_csv_files'
    file = f'{state}.csv'
    file_path = os.path.join(csv_files_dir, file)
    df = read_csv_pl(file_path, separator = ',')
    df = df.to_pandas()
    return df

Let's read the data corresponding to a single state ('AC' in this case) to explore it.

In [ ]:
csv_files_dir = '../data/csv_files'
file = 'RS.csv'
file_path = os.path.join(csv_files_dir, file)
df = read_csv_pl(file_path, separator = ',')
pdf = df.to_pandas()

Let's check the different types of 'ClassInfraFisica' present.

In [ ]:
df.group_by('ClassInfraFisica').count()

Let's fix the typo:

In [ ]:
df = strip_column_pl(df, 'ClassInfraFisica')

In [ ]:
df.group_by('ClassInfraFisica').agg(pl.col('ClassInfraFisica').count().alias('Count'))

In [ ]:
df = replace_values_pl(df, 'ClassInfraFisica', 'Greenfild', 'Greenfield')

## Investigating the relationship between ClassInfraFisica and NumEstacao

Now, we have to check whether each station (indicated by the column 'NumEstacao') corresponds to a single type of 'ClassInfraFisica'; which is the expected behavior (assumption).

In [ ]:
count_unique_values = df.group_by('NumEstacao').agg(pl.col('ClassInfraFisica').n_unique().alias('unique_count'))
instances_with_different_values = count_unique_values.filter(count_unique_values['unique_count'] > 1)
print(instances_with_different_values)

We found that we can actually get two different 'ClassInfraFisica' for any individual station. But if we check further, we'll see that it's actually pairing types with 'null' values, which does not compromise the assumption.

In [ ]:
df.filter(df['NumEstacao'] == 1007447238)[['NumEstacao', 'ClassInfraFisica']]

Let's group by 'NumEstacao', but keeping all the info present in the rows as sets (or as a single value if all rows match)

In [ ]:
dfg = pdf.groupby('NumEstacao').agg(lambda x: set_aggregation(x))
dfg.head(4)

In [ ]:
print(f"Percentage of initial rows kept when grouping = {len(dfg)*100/len(df):.2f}%")

Now we have a dataframe with all the original information, but only 16% of the rows; in which each represents a single station

## Understanding the Columns

Let's understand what the columns are about. This will futurely enable us to better make sense of them, process them if necessary, or drop irrelevant ones.

From [Página de Emissões - Anatel](https://sistemas.anatel.gov.br/anexar-api/publico/anexos/download/9cfc11fc83fcfc2d2586cdb887f72cb5), we can understand what each column refers to.

Status.state: Situação da solicitação na Anatel.

NumFistel: É um código numérico definido pela Anatel, composto de onze dígitos que identifica a autorização que a empresa possui para determinado serviço em determinada região.

NomeEntidade: Nome da Empresa detentora da estação.

NumServico: Código do serviço de telecomunicações na Anatel.

NumAto: Número do Ato de Autorização de Uso de Radiofrequência, sendo que os quatro últimos dígitos se referem ao ano da assinatura do respectivo ato. Exemplo XYZW2017, ato XYZW de 2017.

EnderecoEstacao: Endereço completo onde a estação está instalada.

EndComplemento: Complemento do endereço, caso haja.

DesignacaoEmissao: Baseado em [Método de Designação - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60403&assuntoPublicacao=null&caminhoRel=null&filtro=1&documentoPath=outros/autocadastramento/metodo_de_designacao.pdf)

Designação de emissão. Para uma completa designação da emissão, necessitamos sempre de 9 caracteres alfanuméricos. Os quatros primeiros representam a _largura de faixa necessária_; os três seguintes as _características básicas_ e os dois últimos as _características adicionais facultativas_.
- Largura de faixa necessária: Para uma dada classe de emissão, o valor mínimo da largura de faixa ocupada pela emissão, suficiente para garantir a transmissão da informação com a velocidade de transmissão e com a qualidade requerida para o sistema empregado, nas condições especificadas. A largura de faixa necessária será sempre expressa por meio de três algarismos, que indicam os três primeiros algarismos significativos de largura de faixa necessária e uma letra que ocupa a posição da virgula decimal e representa a Unidade de largura de faixa e será H para Hertz, K para Kilohertz, M para Megahertz ou G para Gigahertz.
- Características básicas: descritas por três símbolos:
    - Primeiro símbolo: tipo de modulação da portaria principal.
    - Segundo símbolo: natureza do(s) sinais que modulam a portaria principal.
    - Terceiro símbolo: tipo de informação a ser transmitida.
- Características adicionais facultativas: Para descrição mais completa de uma emissão, são previstas duas características facultativas, as quais são expressas pelos quarto e quinto símbolos. Quando não se utiliza o quarto ou o quinto símbolo, convém indicar isso mediante um traço no lugar em que cada símbolo apareceria.

meioAcesso: é um campo de ‘estações dispensadas de licenciamento’. As opções são: fibra, par metálico, cabo coaxial e radiação restrita.

Azimute: posicionamento m graus em relação ao Norte do lóbulo principal de radiação da antena. Quando for utilizada antena omnidirecional será 0.

CodTipoClasseEstacao: Classe da Estação conforme lista permitida no sistema de canalização do Mosaico, conforme o Regulamento da faixa de frequência utilizada conforme Manual de Projetos Técnicos do SITAR, disponível em [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- ML: Estação móvel terrestre
- FX: Estação fixa
- FB: Estação de base
- XR: Estação fixa repetidora
- FA: Estação aeronáutica
- RC: Radiofarol não direcional

ClassInfraFisica: Classificação de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (STFC, SCM e SMP). Para serviços de interesse restrito (SLP), estará em branco. Valores válidos: Greenfield, Streetlevel, Ran Sharing, Small Cell, COW, Picocelula, Harmonizada, Rooftop, Indoor, Outdoor

CompartilhamentoInfraFisica: Compartilhamento de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (não se aplica para SLP).

CodTipoAntena: Tipo Antena: campo obrigatório, onde é inserido o código conforme item 2.3.4 do [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- 019: Monopolo vertical
- 060: V invertido
- 213: Cabo fendido

GanhoAntena: Ganho da Antena em dB. Se a frequência for superior a 28.000 kHz, é informado o ganho da antena em relação a uma antena isotrópica (dbi); se inferior, este é dado em relação a uma antena dipolo (dBd).

FrenteCostaAntena: Relação Frente/Costa em dBi (campo obrigatório). Deve ser menor que 90 dbi. Será 0 (zero) quando for utilizada antena omnidirecional.

AnguloMeiaPotenciaAntena: Ângulo de meia potência em graus decimais (campo obrigatório). Pontos no diagrama onde a potência radiada equivale à metade da radiada na direção principal.

AnguloElevacao: : Ângulo de elevação mecânico em graus decimais. O valor deve estar entre 0 e 90 e ser medido com relação à linha de horizonte, sendo negativo quando a linha de visada estiver abaixo desta referência.

Polarizacao:  a orientação do campo elétrico da onda de rádio com respeito a terra ou direção de propagação; e é determinado pela estrutura física da antena e por sua orientação. Valores aceitos: H (horizontal), V (vertical), CR (circular á direita), CL (circular à esquerda) e X (horizontal e vertical simultâneas ou não).

AlturaAntena: Altura da antena (em relação a cota do terreno). Deve ser menor que 200 m.

PotenciaTransmissorWatts: Potência nominal (saída do transmissor) em watts.

CodDebitoTFI: Código da Taxa de Fiscalização de Instalação (TFI) conforme lei nº 5.070, de 7 de julho de 1966.
Taxa de fiscalização da instalação é aquela devida pelas concessionárias e permissionárias de serviços de telecomunicações, no momento em que lhes é outorgada autorização para a execução do serviço e tem a finalidade de ressarcir as despesas realizadas pelo Poder Público até o licenciamento das respectivas estações. Não serão licenciadas as estações das permissionárias e concessionárias de serviços de telecomunicações que não efetuarem o pagamento da taxa de fiscalização da instalação.
[Lei do Fistel](https://www2.camara.leg.br/legin/fed/lei/1960-1969/lei-5070-7-julho-1966-364619-anexo-pl.pdf)

DataLicenciamento:  Data do último licenciamento da estação. Formato: AAAA/MM/DD

DataPrimeiroLicenciamento: Data do primeiro licenciamento da estação. Formato: AAAA/MM/DD

NumRede: Valor numérico, definido pelo usuário, para identificar grupo de estações que se comunicam. Visa facilitar a análise da comunicação das redes.

_id:  Identificação da emissão / frequência da estação

DataValidade: Validade da RF associada àquela estação.

NumFistelAssociado: Campo preenchido quando a emissão for de outro Fistel (do mesmo CNPJ ou diferentes): licenciamento conjunto. Nesse caso, é informado o número do Fistel responsável pelo Ato de RF da emissão dessa linha



meioAcesso consists of all None values

In [ ]:
pdf['meioAcesso'].value_counts(dropna=False)

In [ ]:
# CodEquipamentoTransmissor
print(f"number of rows: {len(pdf['CodEquipamentoTransmissor'])}")
print(f"number of unique rows: {pdf['CodEquipamentoTransmissor'].nunique()}")

In [ ]:
# CodDebitoTFI
pdf['CodDebitoTFI'].value_counts()

In [ ]:
# _id
pdf['_id'].value_counts()

In [ ]:
# DataValidade
pdf['DataValidade'].value_counts()

In [ ]:
pdf['NumFistelAssociado'].value_counts()

In [ ]:
pdf['NomeEntidadeAssociado'].value_counts()

## Filtering out irrelevant columns

In [ ]:
pdf = filter_columns(pdf)

pdf.head()

### NomeEntidade

Given the high number of different providers found in this column (sometimes it's the mobile network operator, sometimes it's the venue owner), we will not be using this column.

## Column Treatment

Now we will correctly treat each column in order to have relevant information and prepare it for training.

### SiglaUf

In [ ]:
pdf['SiglaUf']

### CodMunicipio

In [ ]:
pdf['CodMunicipio'] = pdf['CodMunicipio'].fillna(0).astype(int)
pdf['CodMunicipio']

### DesignacaoEmissao

In [ ]:
pdf[['LarguraFaixaNecessaria', 'CaracteristicasBasicas']] = pdf.apply(process_designacao_emissao, axis=1)
pdf.drop(columns=['DesignacaoEmissao'], inplace=True)

### Tecnologia & tipoTecnologia

In [ ]:
pdf['Tecnologia_e_Tipo'] = pdf.apply(lambda row: concatenate_columns(row, 'Tecnologia', 'tipoTecnologia'), axis=1)
pdf.drop(columns=['Tecnologia', 'tipoTecnologia'], inplace=True)

In [ ]:
x[~x['tipoTecnologia'].isnull()]

### FreqTxMHz

In [ ]:
pdf['FreqTxMHz']

### FreqRxMHz

In [ ]:
pdf['FreqRxMHz']

### CodTipoClasseEstacao

In [ ]:
pdf['CodTipoClasseEstacao'].value_counts()

### ClassInfraFisica

In [ ]:
file = 'SP.csv'
file_path = os.path.join(csv_files_dir, file)
df = read_csv_pl(file_path, separator = ',')
pdf = df.to_pandas()
pdf = strip_column(pdf, 'ClassInfraFisica')
pdf['ClassInfraFisica'] = pdf['ClassInfraFisica'].str.upper()
pdf['ClassInfraFisica'].value_counts(dropna=False)

In [ ]:
pdf['ClassInfraFisica'].value_counts(dropna=False)

In [ ]:
pdf = strip_column(pdf, 'ClassInfraFisica')
pdf['ClassInfraFisica'] = pdf['ClassInfraFisica'].str.upper()
pdf = replace_values(pdf, 'ClassInfraFisica', 'GREENFILD', 'GREENFIELD')
pdf['ClassInfraFisica'].value_counts(dropna=False)

### CompartilhamentoInfraFisica

In [ ]:
pdf['CompartilhamentoInfraFisica'].value_counts(dropna=False)

### CodTipoAntena

In [ ]:
pdf['CodTipoAntena'].value_counts()

In [ ]:
pdf['CodTipoAntena'] = pdf['CodTipoAntena'].fillna(0).astype(int)
pdf['CodTipoAntena'] = pdf['CodTipoAntena'].astype(str)
pdf['CodTipoAntena'].value_counts()

### GanhoAntena

In [ ]:
pdf['GanhoAntena'].value_counts()

In [ ]:
pdf['GanhoAntena'] = pd.to_numeric(pdf['GanhoAntena'], errors='coerce')
pdf['GanhoAntena']

### FrenteCostaAntena

In [ ]:
pdf['FrenteCostaAntena'] = pd.to_numeric(pdf['FrenteCostaAntena'], errors='coerce')
pdf['FrenteCostaAntena']

### AnguloMeiaPotenciaAntena

In [ ]:
pdf['AnguloMeiaPotenciaAntena'] = pd.to_numeric(pdf['AnguloMeiaPotenciaAntena'], errors='coerce')
pdf['AnguloMeiaPotenciaAntena']

### AnguloElevacao

In [ ]:
pdf['AnguloElevacao'] = pd.to_numeric(pdf['AnguloElevacao'], errors='coerce')
pdf['AnguloElevacao']

### Polarizacao

In [ ]:
pdf['Polarizacao'] = pdf['Polarizacao'].str.upper()
pdf['Polarizacao']

### AlturaAntena

In [ ]:
pdf['AlturaAntena'] = pd.to_numeric(pdf['AlturaAntena'], errors='coerce')
pdf['AlturaAntena']

### CodEquipamentoTransmissor

In [ ]:
pdf['CodEquipamentoTransmissor'].value_counts()

### PotenciaTransmissorWatts

In [ ]:
pdf['PotenciaTransmissorWatts'].value_counts(dropna=False)

### CodDebitoTFI

In [ ]:
pdf['CodDebitoTFI'].value_counts(dropna=False)

### DataLicenciamento

In [ ]:
# pdf['DataLicenciamento'] = pd.to_datetime(pdf['DataLicenciamento'], errors='coerce')
# pdf['IdadeLicenciamento'] = (pd.to_datetime('today') - pdf['DataLicenciamento']).dt.days
# pdf['IdadeLicenciamento'].where(pdf['DataLicenciamento'].notna(), None, inplace=True)
pdf['IdadeLicenciamento'] = pdf.apply(process_data, date_column='DataLicenciamento', axis=1)

In [ ]:
# pdf[pdf['DataLicenciamento'].isna()][['DataLicenciamento', 'IdadeLicenciamento']]
z = pdf[['DataLicenciamento', 'IdadeLicenciamento']]

### DataPrimeiroLicenciamento

In [ ]:
pdf['IdadePrimeiroLicenciamento'] = pdf.apply(process_data, date_column='DataPrimeiroLicenciamento', axis=1)

In [ ]:
z = pdf[['NumEstacao', 'DataLicenciamento', 'IdadeLicenciamento', 'DataPrimeiroLicenciamento', 'IdadePrimeiroLicenciamento']]

### DataValidade

In [ ]:
pdf['DiasAteExpirar'] = pdf.apply(process_data, date_column='DataValidade', axis=1)

In [ ]:
pdf[['DiasAteExpirar', 'DataValidade']]

### NumFistelAssociado

Huge number of NaNs and not that important of a column

In [ ]:
pdf['NumFistelAssociado'].value_counts(dropna=False)

# Labeled ClassInfraFisica proportion

In [22]:
df = get_cleaned_df('PR')

In [23]:
dfg = df.groupby('NumEstacao').agg(lambda x: set_aggregation(x))

In [4]:
z = dfg[['ClassInfraFisica']]

In [24]:
x = dfg.isna().sum().reset_index(name='NaN_Count').merge(dfg.count().reset_index(name='Non_NaN_Count'), how='outer')
x['NaN_proportion'] = 100*x['NaN_Count']/(x['NaN_Count'] + x['Non_NaN_Count'])
x[x['index']=='ClassInfraFisica']

,index,NaN_Count,Non_NaN_Count,NaN_proportion
5,ClassInfraFisica,54945,3667,93.743602


# Defining Grouping Criterion for Each Column

In [29]:
df = get_cleaned_df('PR')
df

,NumEstacao,SiglaUf,CodMunicipio,FreqTxMHz,FreqRxMHz,CodTipoClasseEstacao,ClassInfraFisica,CompartilhamentoInfraFisica,CodTipoAntena,GanhoAntena,...,CodEquipamentoTransmissor,PotenciaTransmissorWatts,CodDebitoTFI,DataValidade,LarguraFaixaNecessaria,CaracteristicasBasicas,Tecnologia_e_Tipo,DiasDesdeLicenciamento,DiasDesdePrimeiroLicenciamento,DiasAteExpirar
0,211222,PR,4115200,2150.0,1960.0,FB,None,NA,760,15.86,...,2.246100e+10,40.0,G,2029-12-08,{'5M00'},['G9W'],WCDMA,133.0,7217.0,-2210
1,211222,PR,4115200,1825.0,1730.0,FB,None,NA,760,15.86,...,8.307180e+10,20.0,G,2029-12-08,{'200K'},['G7W'],GSM,133.0,7217.0,-2210
2,211222,PR,4115200,1825.0,1730.0,FB,None,NA,760,15.86,...,8.307180e+10,20.0,G,2029-12-08,{'200K'},['G7W'],GSM,133.0,7217.0,-2210
3,211222,PR,4115200,1825.0,1730.0,FB,None,NA,760,15.86,...,8.307180e+10,20.0,G,2029-12-08,{'200K'},['G7W'],GSM,133.0,7217.0,-2210
4,211222,PR,4115200,1832.5,1737.5,FB,None,NA,760,15.86,...,8.307180e+10,20.0,G,2029-12-08,{'200K'},['G7W'],GSM,133.0,7217.0,-2210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383571,1001101992,PR,4115200,2655.0,2535.0,FB,None,None,760,16.15,...,5.669190e+10,40.0,G,2038-04-30,{'10M0'},['G7W'],LTE,45.0,3191.0,-5275
383572,1001101992,PR,4115200,2655.0,2535.0,FB,None,None,760,16.15,...,5.669190e+10,40.0,G,2038-04-30,{'10M0'},['G7W'],LTE,45.0,3191.0,-5275
383573,692663517,PR,4115200,2655.0,2535.0,FB,None,None,760,16.15,...,5.669190e+10,40.0,G,2038-04-30,{'10M0'},['G7W'],LTE,45.0,4781.0,-5275
383574,692663517,PR,4115200,2655.0,2535.0,FB,None,None,760,16.15,...,5.669190e+10,40.0,G,2038-04-30,{'10M0'},['G7W'],LTE,45.0,4781.0,-5275
